Übung: Verteilung 
-----------------

In dieser Übung erstellen wir mehrere Pods (Instanzen) ab dem gleichen Image (Applikation) mit jeweils einem ReplicaSet, Deployment und Service.

Das passiert in einer eigenen Namespace um die Resultate gezielt Darstellen zu können:

In [ ]:
! kubectl create namespace rs

Erzeugen eines Deployments, hier der das Beispiel von Docker mit einem Web Server welche die aktuelle IP-Adresse ausgibt.

In [ ]:
! # kubectl run webshop --image=registry.gitlab.com/ch-mc-b/autoshop/shop:2.0.0 --namespace rs",
! kubectl create deployment webshop --image=registry.gitlab.com/ch-mc-b/autoshop/shop:2.0.0 --namespace rs

Ausgabe der YAML Datei welche den Erzeugten Ressourcen beschreibt.

Ab `spec.containers` kommt erst der Pod.

In [ ]:
%%bash
kubectl get deployment webshop -n rs -o go-template='
apiVersion: apps/v1
kind: Deployment
metadata:
  name: {{ .metadata.name }}
spec:
  replicas: {{ .spec.replicas }}
  selector:
    matchLabels:
{{- range $k, $v := .spec.selector.matchLabels }}
      {{ $k }}: {{ $v }}
{{- end }}
  template:
    metadata:
      labels:
{{- range $k, $v := .spec.template.metadata.labels }}
        {{ $k }}: {{ $v }}
{{- end }}
    spec:
      containers:
{{- range .spec.template.spec.containers }}
      - name: {{ .name }}
        image: {{ .image }}
        imagePullPolicy: {{ .imagePullPolicy }}
{{- end }}
'

Um die ReplicaSet Funktionalität zu Demonstrieren, setzen wir die Anzahl der laufenden Pods auf 3 und schauen uns das Ergebnis an

In [ ]:
! kubectl --namespace rs scale --replicas=3 deployment/webshop  
# ! kubectl get pod,deployment,replicaset,service --namespace rs
! kube-lineage --namespace rs deployment webshop -d 2 --exclude-types events 

Zu dem **Deployment** `webshop` Erzeugen wir einen Service. Dadurch wird der Web Server von aussen sichtbar.

Der Port 8080 wird von Kubernetes automatisch auf den nächsten freien Port gemappt.

Das Ergebnis schauen wir uns wieder an.

In [ ]:
! kubectl expose deployment/webshop --type="LoadBalancer" --port 8080 --namespace rs
# ! kubectl get pod,deployment,replicaset,service,endpoints -o wide --namespace rs
! kube-lineage --namespace rs deployment webshop -d 2 --exclude-types events && echo ""
! kubectl get service -o wide --namespace rs

Da wir keinen LoadBalancer haben müssen wir mit einem kleinen Shellscript selber die IP des Clusters und der gemappte Port (port-based-routing) als URL aufbereiten.

In [ ]:
! echo "http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace rs webshop -o=jsonpath='{ .spec.ports[0].nodePort }')

Wird der obige Link in mehreren Browsern geöffnet sollte jeweils ein anderer Hostname erscheinen. 

**Erklärung**: Kubernetes verteilt die Anfragen auf die laufenden Pods.

- - - 

Zum Aufräumen genügt es den Namespace zu löschen

In [ ]:
! kubectl delete namespace rs